#### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import re


#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('../files/hr_raw_data.csv', index_col=0)


In [3]:
df.head(3)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1


#### VARIABLES AND FUNCCIONS

In [ ]:
#COLUMN RENAMES:

title_mapping = {"employeenumber": "employee_number",
                "gender": "gender",
                "datebirth": "birth_year",
                "age": "age",
                "maritalstatus": "marital_status",
                "jobrole": "job_title",
                "department": "department",
                "attrition": "terminated",
                "standardhours": "standard_hours",
                "monthlyincome": "monthly_income",
                "remotework": "remote",
                "businesstravel": "business_travel",
                "dailyrate": "daily_rate",
                "distancefromhome": "dist_home",
                "educationfield": "education_field",
                "education": "education_scale",
                "environmentsatisfaction": "env_sat_rate",
                "hourlyrate": "hourly_rate",
                "jobinvolvement": "job_involvement",
                "joblevel": "job_level",
                "jobsatisfaction": "job_sat_rate",
                "monthlyrate": "monthly_rate",
                "numcompaniesworked": "num_comp_worked",
                "overtime": "over_time",
                "percentsalaryhike": "perc_salary_hike",
                "performancerating": "perf_rate",
                "relationshipsatisfaction": "relationship_sat_rate",
                "stockoptionlevel": "stock_opt_level",
                "totalworkingyears": "tot_working_year",
                "trainingtimeslastyear": "traning_times_last_year",
                "worklifebalance": "work_life_balance",
                "yearsatcompany": "year_at_comp",
                "yearsincurrentrole": "year_current_role",
                "yearssincelastpromotion": "year_last_promotion",
                "yearswithcurrmanager": "year_current_mngr",
                "salary": "annual_salary",
                "roledepartament": "role_department"}

#CATEGORIES: 

columns_personal =  ['employee_number', 
                    'gender', 
                    'birth_year', 
                    'age', 
                    'marital_status',
                    'dist_home']

columns_job =   ['job_title',
                 'department',
                'terminated',
                'year_at_comp',
                'year_current_role',
                'standard_hours',
                'remote',
                'business_travel',
                'over_time', 
                'job_level', 
                'stock_opt_level', 
                'traning_times_last_year', 
                'perf_rate',
                'year_last_promotion',
                'year_current_mngr']

columns_education = ['education_field',
                    'education_scale']


columns_income =    ['annual_salary',
                    'monthly_income',
                    'daily_rate',
                    'hourly_rate',
                    'monthly_rate',
                    'perc_salary_hike']

columns_satisfaction =  ['env_sat_rate',
                        'job_involvement',
                        'job_sat_rate',
                        'relationship_sat_rate',
                        'work_life_balance']

columns_emp_bgd =   ['num_comp_worked',
                    'tot_working_year']

drop_colums = ['year_current_role', 'roledepartmet']

#COLUMN REORDER:

new_order_columns = columns_personal+columns_job+columns_education+columns_income+columns_satisfaction+columns_emp_bgd

def reorder_columns(df, list_columns):

    #Reorder columns as per provided list, add all the missing ones at the end.
    
    try:
        df = df[new_order_columns]
        extra_columns = [col for col in df.columns if col not in new_order_columns]
        df = df[ new_order_columns + extra_columns ]
    
    except KeyError as e:
        print(f"KeyError: {e}")
        missing_columns = [col for col in new_order_columns if col not in df.columns]
        print(f"Missing columns: {missing_columns}")

        extra_columns = [col for col in df.columns if col not in new_order_columns]
        df = df[ new_order_columns + extra_columns ]
    
    return df


#DATA ANALYSIS

def data_analysis (dataframe, columns):

    print('📌CATEGORY ANALYSIS:\n')
    display(df[columns_personal].describe().T)
    print('\n')

    for col in columns:
        
        print(f'📌Columna: {col.upper()} \n')
        print('Null counter:', dataframe[col].isnull().sum(), '\n')
        print('These are the unique values:', dataframe[col].unique(),'\n')
        print('These are values counter:', dataframe[col].value_counts(dropna=False),'\n')
        print('-------------')


#### STRUCTURE CLEANING

In [ ]:
#ELIMINAR DUPLICADOS, NOS QUEDAMOS CON EL PRIMERO
df[df.duplicated(subset='employeenumber', keep='first')].sort_values('employeenumber')

In [ ]:
#Splitting column "roledepartament"
#df[["role_1", "department_1"]]=df["roledepartament"].str.split("-", expand=True).get([0, 1])

KeyError: 'roledepartament'

In [5]:
#RENAME
df = df.rename(columns=title_mapping)

In [6]:
df.head(1)

,age,terminated,business_travel,daily_rate,department,dist_home,education_scale,education_field,employeecount,employee_number,env_sat_rate,gender,hourly_rate,job_involvement,job_level,job_title,job_sat_rate,marital_status,monthly_income,monthly_rate,num_comp_worked,over18,over_time,perc_salary_hike,perf_rate,relationship_sat_rate,standard_hours,stock_opt_level,tot_working_year,traning_times_last_year,work_life_balance,year_at_comp,year_current_role,year_last_promotion,year_current_mngr,sameasmonthlyincome,birth_year,annual_salary,role_department,numberchildren,remote
0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes


In [9]:
#REORDER
df = reorder_columns(df, new_order_columns)

In [13]:
df.head()

,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,terminated,year_at_comp,year_current_role,standard_hours,remote,business_travel,over_time,job_level,stock_opt_level,traning_times_last_year,perf_rate,year_last_promotion,year_current_mngr,education_field,education_scale,annual_salary,monthly_income,daily_rate,hourly_rate,monthly_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,0,1972,51,NaN,6,Research Director,NaN,No,20,NaN,Full Time,Yes,NaN,No,5,0,5,"3,0",15,15,NaN,3,"195370,00$","16280,83$",2015.722222,NaN,"42330,17$",13,1,3,3,3,"3,0",7,NaN
1,2,0,1971,52,NaN,1,Manager,NaN,No,33,NaN,NaN,1,NaN,NaN,5,1,5,"3,0",11,9,Life Sciences,4,"199990,00$",NaN,2063.388889,NaN,"43331,17$",14,3,2,3,1,"3,0",0,"34,0"
2,3,0,1981,42,Married,4,Manager,Research & Development,No,22,NaN,NaN,1,travel_rarely,No,5,0,3,"3,0",11,15,Technical Degree,2,"192320,00$",NaN,1984.253968,NaN,"41669,33$",11,3,3,4,4,NaN,1,"22,0"
3,4,1,1976,47,Married,2,Research Director,NaN,No,20,NaN,Full Time,False,travel_rarely,NaN,4,2,2,"3,0",5,6,Medical,4,"171690,00$","14307,50$",1771.404762,NaN,"37199,50$",19,1,3,3,2,NaN,3,NaN
4,5,1,1977,46,Divorced,3,Sales Executive,NaN,No,19,NaN,NaN,0,NaN,No,4,1,5,"3,0",2,8,Technical Degree,3,NaN,"12783,92$",1582.771346,NaN,"33238,20$",12,1,4,1,4,"3,0",2,NaN


In [12]:
# Title and strip values

to_title = ['marital_status', 'job_title', 'department', 'education_field']

df[to_title] = df[to_title].apply(lambda x: x.str.title().str.strip() if x.dtype == 'object' else x)

In [14]:
# Update 0/1 from Genger to M/F

gender_map = {0: "M", 1:"F"}
df['gender'] = df['gender'].map(gender_map)

In [15]:
# Replacing values obj to numeric in age

age_update = {
    'thirty-two': 32,
    'twenty-four': 24,
    'thirty': 30,
    'fifty-eight': 58,
    'fifty-two': 52,
    'twenty-six': 26,
    'fifty-five': 55,
    'thirty-seven': 37,
    'thirty-six': 36,
    'forty-seven': 47,
    'thirty-one': 31}

df['age'] = df['age'].replace(age_update)


In [16]:
convert_float = ['annual_salary','monthly_income','daily_rate']
def replace_currency(value):
        try:
                value = float(value.replace("$", "").replace(",", "."))
                return value
        except:
            return value
#df[convert_float] = df[convert_float].apply(replace_currency)




for col in convert_float:
     df[col]=df[col].apply(replace_currency)

In [18]:
df["marital_status"]=df["marital_status"].replace("Marreid", "Married")

In [30]:
df['year_current_role'].unique()

array([nan, '13,0', '12,0', '11,0', '7,0', '6,0', '4,0', '3,0', '2,0',
       '1,0', '0,0'], dtype=object)

In [ ]:
df.isnull().sum() / df.shape[0] * 100 

employee_number             0.000000
gender                      0.000000
birth_year                  0.000000
age                         0.000000
marital_status             40.226460
dist_home                   0.000000
job_title                   0.000000
department                 81.406436
terminated                  0.000000
year_at_comp                0.000000
year_current_role          97.914184
standard_hours             20.917759
remote                      0.000000
business_travel            47.735399
over_time                  41.477950
job_level                   0.000000
stock_opt_level             0.000000
traning_times_last_year     0.000000
perf_rate                  11.918951
year_last_promotion         0.000000
year_current_mngr           0.000000
education_field            46.126341
education_scale             0.000000
annual_salary              16.984505
monthly_income             29.141836
daily_rate                  0.000000
hourly_rate                75.506555
m

In [40]:
df['over_time'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [23]:
df.head()

,employee_number,gender,birth_year,age,marital_status,dist_home,job_title,department,terminated,year_at_comp,year_current_role,standard_hours,remote,business_travel,over_time,job_level,stock_opt_level,traning_times_last_year,perf_rate,year_last_promotion,year_current_mngr,education_field,education_scale,annual_salary,monthly_income,daily_rate,hourly_rate,monthly_rate,perc_salary_hike,env_sat_rate,job_involvement,job_sat_rate,relationship_sat_rate,work_life_balance,num_comp_worked,tot_working_year
0,1,M,1972,51,NaN,6,Research Director,NaN,No,20,NaN,Full Time,Yes,NaN,No,5,0,5,"3,0",15,15,NaN,3,195370.0,16280.83,2015.722222,NaN,"42330,17$",13,1,3,3,3,"3,0",7,NaN
1,2,M,1971,52,NaN,1,Manager,NaN,No,33,NaN,NaN,1,NaN,NaN,5,1,5,"3,0",11,9,Life Sciences,4,199990.0,NaN,2063.388889,NaN,"43331,17$",14,3,2,3,1,"3,0",0,"34,0"
2,3,M,1981,42,Married,4,Manager,Research & Development,No,22,NaN,NaN,1,travel_rarely,No,5,0,3,"3,0",11,15,Technical Degree,2,192320.0,NaN,1984.253968,NaN,"41669,33$",11,3,3,4,4,NaN,1,"22,0"
3,4,F,1976,47,Married,2,Research Director,NaN,No,20,NaN,Full Time,False,travel_rarely,NaN,4,2,2,"3,0",5,6,Medical,4,171690.0,14307.50,1771.404762,NaN,"37199,50$",19,1,3,3,2,NaN,3,NaN
4,5,F,1977,46,Divorced,3,Sales Executive,NaN,No,19,NaN,NaN,0,NaN,No,4,1,5,"3,0",2,8,Technical Degree,3,NaN,12783.92,1582.771346,NaN,"33238,20$",12,1,4,1,4,"3,0",2,NaN


NUMERICAS
KNII:   annual_salary (con monthly_income y daily_rate)
mediana: perf_rate,

CATEGÓRICAS
unknown: marital_status, department (solo manager), business_travel, over_time, education_field,        
knni: department (menos manager), standard_hours (job_title y anual_salary)

*** HACER BARPLOT CON education_field PARA VER SI HAY RELACION CON job_title y education_scale
*** HACER BARPLOT CON tot_working_year  PARA VER SI HAY RELACION CON age y year_at_comp

over_time: la cambiamos por unknown, y vamos a agrupar por yes o no y medir el grado de satisfacción.

NEXT STEPS:
Pedir más datos de over_time porque casi la mitad son nulos y puede haber relación con la satisfacción

salary_base =['monthly_income', 'daily_rate', 'annual_salary']
imputer = KNNImputer(n_neighbors=3)
salary_base = imputer.fit_transform(df[salary_base])

#### DATA ANALYSYS

##### PERSONAL COLUMNS

In [39]:
data_analysis(df, columns_personal)

📌CATEGORY ANALYSIS:



,count,mean,std,min,25%,50%,75%,max
employee_number,1678.0,809.859952,467.084867,1.0,403.25,813.5,1215.75,1614.0
birth_year,1678.0,1986.047080,9.154908,1963.0,1980.00,1987.0,1993.00,2005.0
dist_home,1678.0,4.504172,14.652066,-49.0,2.00,5.0,11.00,29.0




📌Columna: EMPLOYEE_NUMBER 

Null counter: 0 

These are the unique values: [   1    2    3 ... 1612 1613 1614] 

These are values counter: employee_number
300     2
159     2
271     2
1158    2
191     2
       ..
547     1
546     1
545     1
544     1
1614    1
Name: count, Length: 1614, dtype: int64 

-------------
📌Columna: GENDER 

Null counter: 0 

These are the unique values: ['M' 'F'] 

These are values counter: gender
M    1009
F     669
Name: count, dtype: int64 

-------------
📌Columna: BIRTH_YEAR 

Null counter: 0 

These are the unique values: [1972 1971 1981 1976 1977 1975 1964 1982 1967 1985 1968 1983 1965 1988
 1978 1990 1987 1989 1970 1980 1963 1991 1986 1974 1984 1973 1979 1993
 1994 1992 1969 1966 1996 1995 1997 1998 1999 2000 2001 2002 2003 2004
 2005] 

These are values counter: birth_year
1992    89
1988    88
1989    86
1994    82
1987    80
1991    67
1993    66
1985    64
1990    63
1983    60
1986    56
1995    54
1996    54
1978    50
1981    50
1982    48

##### JOB COLUMNS

In [46]:
data_analysis(df, columns_job)

📌CATEGORY ANALYSIS:



,count,mean,std,min,25%,50%,75%,max
employee_number,1678.0,809.859952,467.084867,1.0,403.25,813.5,1215.75,1614.0
birth_year,1678.0,1986.047080,9.154908,1963.0,1980.00,1987.0,1993.00,2005.0
dist_home,1678.0,4.504172,14.652066,-49.0,2.00,5.0,11.00,29.0




📌Columna: JOB_TITLE 

Null counter: 0 

These are the unique values: ['Research Director' 'Manager' 'Sales Executive' 'Manufacturing Director'
 'Research Scientist' 'Healthcare Representative' 'Laboratory Technician'
 'Sales Representative' 'Human Resources'] 

These are values counter: job_title
Sales Executive              381
Research Scientist           325
Laboratory Technician        289
Manufacturing Director       168
Healthcare Representative    155
Manager                      115
Sales Representative          96
Research Director             90
Human Resources               59
Name: count, dtype: int64 

-------------
📌Columna: DEPARTMENT 

Null counter: 1366 

These are the unique values: [nan 'Research & Development' 'Sales' 'Human Resources'] 

These are values counter: department
NaN                       1366
Research & Development     203
Sales                       93
Human Resources             16
Name: count, dtype: int64 

-------------
📌Columna: TERMINATED 

Nul

##### INCOME COLUMNS

In [ ]:
data_analysis(df, columns_income)

##### EDUCATION COLUMNS

In [ ]:
data_analysis(df,columns_education)

##### SATISFACTION COLUMNS

In [44]:
data_analysis(df,columns_satisfaction)

📌CATEGORY ANALYSIS:



,count,mean,std,min,25%,50%,75%,max
employee_number,1678.0,809.859952,467.084867,1.0,403.25,813.5,1215.75,1614.0
birth_year,1678.0,1986.047080,9.154908,1963.0,1980.00,1987.0,1993.00,2005.0
dist_home,1678.0,4.504172,14.652066,-49.0,2.00,5.0,11.00,29.0




📌Columna: ENV_SAT_RATE 

Null counter: 0 

These are the unique values: [ 1  3  4  2 42 37 35 25 27 31 39 21 15 14 33 19 12 13 28 47 36 29 24 46
 16 22 41 49 11 48 18 10 45 38 17 20 26 43] 

These are values counter: env_sat_rate
4     483
3     471
2     316
1     306
35      7
12      7
14      6
13      6
24      5
47      5
36      4
48      4
41      4
46      4
42      4
45      3
11      3
22      3
17      3
18      3
20      3
25      3
27      3
37      2
29      2
19      2
38      2
31      2
15      2
16      2
26      1
39      1
10      1
49      1
21      1
28      1
33      1
43      1
Name: count, dtype: int64 

-------------
📌Columna: JOB_INVOLVEMENT 

Null counter: 0 

These are the unique values: [3 2 4 1] 

These are values counter: job_involvement
3    995
2    421
4    170
1     92
Name: count, dtype: int64 

-------------
📌Columna: JOB_SAT_RATE 

Null counter: 0 

These are the unique values: [3 4 1 2] 

These are values counter: job_sat_rate
4    538
3    49

##### EMPLOYEMENT BACKGROUND COLUMNS

In [45]:
data_analysis(df, columns_emp_bgd)

📌CATEGORY ANALYSIS:



,count,mean,std,min,25%,50%,75%,max
employee_number,1678.0,809.859952,467.084867,1.0,403.25,813.5,1215.75,1614.0
birth_year,1678.0,1986.047080,9.154908,1963.0,1980.00,1987.0,1993.00,2005.0
dist_home,1678.0,4.504172,14.652066,-49.0,2.00,5.0,11.00,29.0




📌Columna: NUM_COMP_WORKED 

Null counter: 0 

These are the unique values: [7 0 1 3 2 4 8 9 5 6] 

These are values counter: num_comp_worked
1    597
0    234
3    174
4    166
2    163
7     88
6     75
5     67
9     59
8     55
Name: count, dtype: int64 

-------------
📌Columna: TOT_WORKING_YEAR 

Null counter: 549 

These are the unique values: [nan '34,0' '22,0' '28,0' '20,0' '21,0' '33,0' '40,0' '18,0' '25,0' '15,0'
 '17,0' '26,0' '16,0' '24,0' '14,0' '23,0' '27,0' '19,0' '11,0' '38,0'
 '37,0' '13,0' '12,0' '29,0' '10,0' '36,0' '35,0' '9,0' '31,0' '32,0'
 '8,0' '7,0' '30,0' '6,0' '5,0' '4,0' '3,0' '2,0' '1,0' '0,0'] 

These are values counter: tot_working_year
NaN     549
10,0    151
6,0      88
8,0      86
9,0      71
5,0      68
7,0      59
4,0      57
1,0      55
12,0     34
3,0      34
11,0     31
13,0     31
14,0     31
16,0     30
20,0     30
15,0     28
18,0     28
21,0     23
17,0     23
2,0      21
22,0     19
19,0     17
28,0     14
24,0     14
23,0     13
0,0       8

In [ ]:
df.head()